# Silver Layer - Nettoyage & Enrichissement

In [4]:
from pyspark.sql import functions as F
# Vérification des tables créées
print("📋 Liste des tables dans le namespace bronze:")
spark.sql("SHOW TABLES IN iceberg.bronze").show()

# Pour une table spécifique
print("\n🔍 Description de la table taxi_trips_2024:")
spark.sql("DESCRIBE EXTENDED iceberg.bronze.taxi_trips_2024").show(truncate=False)

# Vérifier les données
print("\n📊 Aperçu des données (5 premières lignes):")
spark.sql("SELECT * FROM iceberg.bronze.taxi_trips_2024 LIMIT 5").show()

# Vérifier les partitions
print("\n🗂️ Partitions de la table taxi_trips_2024:")
spark.sql("SHOW PARTITIONS iceberg.bronze.taxi_trips_2024").show()

📋 Liste des tables dans le namespace bronze:


Py4JJavaError: An error occurred while calling o37.sql.
: org.apache.iceberg.exceptions.NoSuchNamespaceException: Namespace does not exist: iceberg.bronze
	at org.apache.iceberg.rest.ErrorHandlers$NamespaceErrorHandler.accept(ErrorHandlers.java:173)
	at org.apache.iceberg.rest.ErrorHandlers$NamespaceErrorHandler.accept(ErrorHandlers.java:166)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:224)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:308)
	at org.apache.iceberg.rest.BaseHTTPClient.get(BaseHTTPClient.java:77)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:97)
	at org.apache.iceberg.rest.RESTSessionCatalog.listTables(RESTSessionCatalog.java:388)
	at org.apache.iceberg.catalog.BaseSessionCatalog$AsCatalog.listTables(BaseSessionCatalog.java:79)
	at org.apache.iceberg.rest.RESTCatalog.listTables(RESTCatalog.java:92)
	at org.apache.iceberg.CachingCatalog.listTables(CachingCatalog.java:136)
	at org.apache.iceberg.spark.SparkCatalog.listTa

In [9]:
# notebooks/03_silver_processing_unified_fixed.py

from pyspark.sql import functions as F

def get_bronze_table_name(base_name):
    """
    Trouve automatiquement où se trouve la table Bronze
    """
    possible_locations = [
        f"local.bronze.{base_name}",
        f"iceberg.bronze.{base_name}"
    ]
    
    for table_name in possible_locations:
        try:
            # Tester si la table existe
            spark.sql(f"SELECT 1 FROM {table_name} LIMIT 1")
            print(f"   📍 Table trouvée: {table_name}")
            return table_name
        except:
            continue
    
    raise Exception(f"Table {base_name} non trouvée dans local.bronze ni iceberg.bronze")

def create_silver_tables_unified():
    """
    Version unifiée qui fonctionne avec les deux catalogues - CORRIGÉE
    """
    
    print("🚀 Démarrage Silver Layer (version unifiée)")
    
    # Détecter où sont les tables Bronze
    trips_table = get_bronze_table_name("taxi_trips_2024")
    zones_table = get_bronze_table_name("taxi_zones")
    weather_table = get_bronze_table_name("weather_2024")
    
    # Déterminer où écrire (utiliser le même catalogue que la source)
    catalog = trips_table.split('.')[0]  # "local" ou "iceberg"
    print(f"   📁 Catalogue cible: {catalog}")
    
    # Créer le namespace Silver si nécessaire
    try:
        spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {catalog}.silver")
        print(f"✅ Namespace {catalog}.silver créé/validé")
    except:
        print(f"ℹ️  Namespace {catalog}.silver déjà existant")
    
    # 1. Nettoyage des trips
    print("\n📊 Étape 1: Nettoyage des trips...")
    trips_df = spark.table(trips_table)
    
    print(f"   Lignes brutes: {trips_df.count():,}")
    
    # Nettoyage et enrichissement
    trips_cleaned = trips_df.filter(
        (F.col("trip_distance") > 0) &
        (F.col("trip_distance") < 100) &
        (F.col("total_amount") > 0) &
        (F.col("total_amount") < 500) &
        (F.col("passenger_count") > 0) &
        (F.col("passenger_count") <= 6) &
        (F.col("tpep_dropoff_datetime") > F.col("tpep_pickup_datetime"))
    ).withColumn(
        "trip_duration_minutes",
        (F.unix_timestamp("tpep_dropoff_datetime") - 
         F.unix_timestamp("tpep_pickup_datetime")) / 60
    ).withColumn(
        "speed_mph",
        F.when(
            (F.col("trip_duration_minutes") > 0) & (F.col("trip_distance") > 0),
            F.col("trip_distance") / (F.col("trip_duration_minutes") / 60)
        ).otherwise(None)
    ).withColumn(
        "is_weekend",
        F.when(F.dayofweek("tpep_pickup_datetime").isin([1, 7]), 1).otherwise(0)
    ).withColumn(
        "hour_of_day", F.hour("tpep_pickup_datetime")
    ).withColumn(
        "year", F.year("tpep_pickup_datetime")
    ).withColumn(
        "month", F.month("tpep_pickup_datetime")
    ).withColumn(
        "pickup_date", F.to_date("tpep_pickup_datetime")
    ).withColumn(
        "pickup_time", F.date_format("tpep_pickup_datetime", "HH:mm:ss")
    )
    
    # Filtrer les valeurs aberrantes supplémentaires
    trips_cleaned = trips_cleaned.filter(
        (F.col("trip_duration_minutes") > 0) &
        (F.col("trip_duration_minutes") < 180) &
        (F.col("speed_mph") <= 80) &
        (F.col("speed_mph") >= 0)
    )
    
    print(f"   Lignes après nettoyage: {trips_cleaned.count():,}")
    
    # Écrire dans Silver
    trips_cleaned.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{catalog}.silver.trips_enriched")
    
    print(f"✅ {catalog}.silver.trips_enriched créée: {trips_cleaned.count():,} lignes")
    
    # 2. Enrichissement avec zones
    print("\n🗺️  Étape 2: Enrichissement avec zones...")
    zones_df = spark.table(zones_table)
    
    # Jointure avec zones de pickup
    trips_with_zones = trips_cleaned.alias("t").join(
        zones_df.alias("pz"),
        F.col("t.PULocationID") == F.col("pz.LocationID"),
        "left"
    ).select(
        "t.*",
        F.col("pz.zone").alias("pickup_zone"),
        F.col("pz.borough").alias("pickup_borough")
    )
    
    # Jointure avec zones de dropoff
    trips_with_zones = trips_with_zones.alias("t").join(
        zones_df.alias("dz"),
        F.col("t.DOLocationID") == F.col("dz.LocationID"),
        "left"
    ).select(
        "t.*",
        F.col("dz.zone").alias("dropoff_zone"),
        F.col("dz.borough").alias("dropoff_borough")
    )
    
    # Calculer des métriques supplémentaires
    trips_with_zones = trips_with_zones.withColumn(
        "is_same_borough",
        F.when(
            (F.col("pickup_borough").isNotNull()) & 
            (F.col("dropoff_borough").isNotNull()) &
            (F.col("pickup_borough") == F.col("dropoff_borough")),
            1
        ).otherwise(0)
    ).withColumn(
        "is_airport_trip",
        F.when(
            (F.col("pickup_zone").contains("Airport")) |
            (F.col("dropoff_zone").contains("Airport")) |
            (F.col("PULocationID").isin([1, 132, 138])) |
            (F.col("DOLocationID").isin([1, 132, 138])),
            1
        ).otherwise(0)
    )
    
    trips_with_zones.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{catalog}.silver.trips_with_zones")
    
    print(f"✅ {catalog}.silver.trips_with_zones créée: {trips_with_zones.count():,} lignes")
    
    # 3. Enrichissement avec météo
    print("\n🌤️  Étape 3: Enrichissement avec météo...")
    weather_df = spark.table(weather_table)
    
    # Ajouter l'heure à la météo
    weather_with_hour = weather_df.withColumn(
        "weather_hour", F.hour("time")
    ).withColumn(
        "weather_date", F.to_date("time")
    )
    
    # Jointure météo - CORRECTION ICI
    # On utilise un alias pour le DataFrame de gauche et on sélectionne explicitement les colonnes
    trips_for_weather = trips_with_zones.withColumn(
        "pickup_hour", F.hour("tpep_pickup_datetime")
    )
    
    # Faire la jointure avec alias
    trips_complete = trips_for_weather.alias("t").join(
        weather_with_hour.alias("w"),
        (F.col("t.pickup_date") == F.col("w.weather_date")) &
        (F.col("t.pickup_hour") == F.col("w.weather_hour")),
        "left"
    )
    
    # Maintenant, sélectionner les colonnes explicitement
    # D'abord, obtenir toutes les colonnes du DataFrame de gauche
    left_columns = [F.col(f"t.{col}") for col in trips_for_weather.columns]
    
    # Sélectionner toutes les colonnes de gauche + les colonnes de météo qu'on veut
    trips_complete = trips_complete.select(
        *left_columns,
        F.col("w.temp").alias("temperature"),
        F.col("w.rhum").alias("humidity"),
        F.col("w.prcp").alias("precipitation"),
        F.col("w.wspd").alias("wind_speed"),
        F.col("w.pres").alias("pressure")
    )
    
    # Nettoyer les colonnes temporaires
    trips_complete = trips_complete.drop("pickup_hour")
    
    # Ajouter des features météo dérivées
    trips_complete = trips_complete.withColumn(
        "is_rainy",
        F.when(F.col("precipitation") > 0.1, 1).otherwise(0)
    ).withColumn(
        "is_cold",
        F.when(F.col("temperature") < 32, 1).otherwise(0)  # < 0°C
    ).withColumn(
        "is_hot",
        F.when(F.col("temperature") > 86, 1).otherwise(0)   # > 30°C
    )
    
    # Statistiques de jointure
    total_trips = trips_complete.count()
    trips_with_weather = trips_complete.filter(F.col("temperature").isNotNull()).count()
    
    print(f"   → Trips avec données météo: {trips_with_weather:,}/{total_trips:,} ({(trips_with_weather/total_trips*100):.2f}%)")
    
    trips_complete.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{catalog}.silver.trips_complete")
    
    print(f"✅ {catalog}.silver.trips_complete créée: {trips_complete.count():,} lignes")
    
    # 4. Créer des tables de résumé
    print("\n📈 Étape 4: Création des tables de résumé...")
    
    # Résumé quotidien
    daily_summary = trips_complete.groupBy(
        "pickup_date",
        "pickup_borough",
        "is_weekend"
    ).agg(
        F.count("*").alias("trip_count"),
        F.sum("total_amount").alias("total_revenue"),
        F.avg("trip_distance").alias("avg_distance"),
        F.avg("trip_duration_minutes").alias("avg_duration"),
        F.avg("total_amount").alias("avg_fare"),
        F.sum("passenger_count").alias("total_passengers"),
        F.avg("temperature").alias("avg_temperature"),
        F.avg("precipitation").alias("avg_precipitation")
    ).withColumn(
        "revenue_per_trip",
        F.col("total_revenue") / F.col("trip_count")
    )
    
    daily_summary.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("pickup_date") \
        .saveAsTable(f"{catalog}.silver.daily_summary")
    
    print(f"✅ {catalog}.silver.daily_summary créée")
    
    # Afficher un résumé
    print("\n" + "="*60)
    print("📊 RÉSUMÉ SILVER LAYER")
    print("="*60)
    
    tables = [
        (f"{catalog}.silver.trips_enriched", trips_cleaned.count()),
        (f"{catalog}.silver.trips_with_zones", trips_with_zones.count()),
        (f"{catalog}.silver.trips_complete", trips_complete.count()),
        (f"{catalog}.silver.daily_summary", daily_summary.count())
    ]
    
    for table_name, count in tables:
        print(f"   📁 {table_name}: {count:,} lignes")
    
    # Aperçu des données
    print("\n👁️  Aperçu des données enrichies:")
    trips_complete.select(
        "pickup_date",
        "pickup_zone",
        "dropoff_zone",
        "trip_distance",
        "trip_duration_minutes",
        "total_amount",
        "temperature",
        "is_rainy"
    ).limit(5).show()
    
    print("\n🎉 PIPELINE SILVER TERMINÉ AVEC SUCCÈS!")
    
    return trips_complete

# Exécuter
create_silver_tables_unified()

🚀 Démarrage Silver Layer (version unifiée)


Exception: Table taxi_trips_2024 non trouvée dans local.bronze ni iceberg.bronze

In [3]:
def quick_check_silver():
    """Vérification rapide des tables Silver"""
    
    print("🔍 VÉRIFICATION RAPIDE SILVER")
    
    # Essayer les deux catalogues
    for catalogue in ["local", "iceberg"]:
        print(f"\n🔍 Vérification du catalogue: {catalogue}")
        
        try:
            tables = spark.sql(f"SHOW TABLES IN {catalogue}.silver").collect()
            
            if not tables:
                print(f"   ❌ Aucune table dans {catalogue}.silver")
                continue
            
            print(f"   ✅ Tables trouvées dans {catalogue}.silver:")
            
            for table in tables:
                table_name = table['tableName']
                full_name = f"{catalogue}.silver.{table_name}"
                
                try:
                    count = spark.sql(f"SELECT COUNT(*) as cnt FROM {full_name}").collect()[0]['cnt']
                    print(f"      • {table_name}: {count:,} lignes")
                    
                    # Afficher le schéma pour trips_complete
                    if table_name == "trips_complete":
                        df = spark.table(full_name)
                        print(f"      → Colonnes: {len(df.columns)}")
                        print(f"      → Exemple: {df.columns[:5]}...")
                        
                except Exception as e:
                    print(f"      • {table_name}: Erreur - {e}")
            
            return catalogue  # Retourne le premier catalogue avec des tables
            
        except Exception as e:
            print(f"   ❌ Erreur avec {catalogue}: {e}")
    
    return None

# Exécution
catalogue = quick_check_silver()

if catalogue:
    print(f"\n✅ Catalogue Silver actif: {catalogue}")
    
    # Afficher un échantillon
    print(f"\n👁️  Aperçu de {catalogue}.silver.trips_complete:")
    spark.table(f"{catalogue}.silver.trips_complete").limit(5).show()
    
    # Vérifier les données manquantes
    trips_df = spark.table(f"{catalogue}.silver.trips_complete")
    print(f"\n🔍 Vérification des données manquantes:")
    
    for col in ["pickup_zone", "dropoff_zone", "temperature"]:
        null_count = trips_df.filter(F.col(col).isNull()).count()
        total = trips_df.count()
        percentage = (null_count / total) * 100 if total > 0 else 0
        print(f"   • {col}: {null_count:,} NULL ({percentage:.1f}%)")
    
else:
    print("\n❌ Aucune table Silver trouvée!")

🔍 VÉRIFICATION RAPIDE SILVER

🔍 Vérification du catalogue: local
   ✅ Tables trouvées dans local.silver:
      • daily_summary: 2,487 lignes
      • trips_complete: 35,607,955 lignes
      → Colonnes: 42
      → Exemple: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance']...
      • trips_enriched: 35,595,953 lignes
      • trips_with_zones: 35,607,955 lignes

✅ Catalogue Silver actif: local

👁️  Aperçu de local.silver.trips_complete:
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----+-----+---+---------------------+------------------+----------+-----------+-----------+-----------+-------------------+--------------+--------------------+---------------+---------------+---------------+-----------+--------+-------------+---

   • pickup_zone: 114,033 NULL (0.3%)


   • dropoff_zone: 302,365 NULL (0.8%)
   • temperature: 820,984 NULL (2.3%)


🚀 DÉMARRAGE DU PIPELINE GOLD - ANALYTICS & ML

🚀 DÉMARRAGE DU PIPELINE GOLD - ANALYTICS & ML
🔍 Détection du catalogue Silver...
   ✅ Catalogue trouvé: local

📁 Catalogue Silver détecté: local
✅ Namespace local.gold créé/validé

📊 Création des tables analytiques Gold...

   📅 Création de daily_metrics...


   ✅ Table gold.daily_metrics créée: 374 lignes

   🕐 Création de hourly_patterns...


   ✅ Table gold.hourly_patterns créée: 8,796 lignes

   🗺️  Création de top_routes...


   ✅ Table gold.top_routes créée: 1,000 lignes

   🚕 Création de driver_performance...


   ✅ Table gold.driver_performance créée: 740 lignes

   🌤️  Création de weather_impact...


...

   ✅ Dataset prix créé: 3,560,139 lignes
   ✅ Dataset durée créé: 3,560,139 lignes
   ✅ Dataset pourboire créé: 3,506,557 lignes

📊 Étape 4: Création des tables pour dashboards...

   1. Table: executive_dashboard...
      ✅ Table executive_dashboard créée

   2. Table: hourly_analysis...
      ✅ Table hourly_analysis créée

   3. Table: driver_analytics...
      ✅ Table driver_analytics créée

   4. Table: weather_dashboard...
      ✅ Table weather_dashboard créée

   5. Table: route_analysis...
      ✅ Table route_analysis créée

   6. Table: payment_dashboard...
      ✅ Table payment_dashboard créée

   🎯 Toutes les tables dashboard ont été créées!
      • executive_dashboard
      • hourly_analysis
      • driver_analytics
      • weather_dashboard
      • route_analysis
      • payment_dashboard

🔍 Étape 5: Vérification de la qualité des données Gold...
   ✅ Table daily_business_metrics existe
   ✅ Table hourly_patterns existe
   ✅ Table top_routes existe
   ✅ Table driver_perform

   • daily_metrics: 374 lignes


   • hourly_patterns: 8,796 lignes


   • top_routes: 1,000 lignes


   • driver_performance: 740 lignes
   • ml_dataset: 3,560,139 lignes
   • ml_fare_prediction: 3,560,139 lignes
   • ml_duration_prediction: 3,560,139 lignes
   • ml_tip_prediction: 3,506,557 lignes

🎯 QUALITÉ DES DONNÉES:
   ✅ Tables Gold vérifiées: 6/6

📊 TABLES DASHBOARD CRÉÉES:
   • executive_dashboard
   • hourly_analysis
   • driver_analytics
   • weather_dashboard
   • route_analysis
   • payment_dashboard

✨ PRÊT POUR L'ANALYSE AVANCÉE!
Vous pouvez maintenant:
1. Explorer les tables Gold avec SQL
2. Créer des dashboards avec les tables dashboard
3. Entraîner des modèles ML avec les datasets préparés
4. Analyser les performances des conducteurs
5. Étudier l'impact de la météo sur les trajets

✨ Prêt pour l'analyse avancée! Vous pouvez maintenant:
   1. Explorer les tables Gold avec SQL
   2. Connecter Superset aux vues créées
   3. Lancer des modèles ML sur les datasets préparés
   4. Analyser les métriques business dans les dashboards
